In [ ]:
# -*- coding: utf-8 -*-
"""
特征重要性对比分析脚本
=================================================
本脚本用于加载之前训练并保存的三个最终模型，并对它们的特征重要性进行对比分析和可视化。

**请确保在运行此脚本前，已经成功运行了以下三个脚本：**
1. WGAN-GP + XGBoost 版本
2. SMOTE + XGBoost 版本
3. 无数据增强版本

本脚本将执行以下操作：
1. 定义模型和数据文件的路径。
2. 加载三个已保存的XGBoost模型。
3. 加载原始数据以获取特征名称。
4. 提取每个模型的特征重要性（类型：'gain'）。
5. 将结果合并到一个DataFrame中并打印。
6. 使用条形图对前10个最重要的特征进行可视化对比。
"""

# --- 基础库导入 ---
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import os
import joblib
from pathlib import Path

# --- 0. 全局配置 ---

# --- 路径配置 ---
# 确保这里的路径与您之前脚本的输出路径完全一致
CURRENT_DIR = Path.cwd()
PROJECT_ROOT = CURRENT_DIR.parent
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = PROJECT_ROOT / "output"

# 数据文件路径 (仅用于获取列名)
DEV_SET_FILE = DATA_DIR / "development_set.xlsx"

# 三个实验的输出根目录
WGAN_GP_OUTPUT_DIR = OUTPUT_DIR
SMOTE_OUTPUT_DIR = OUTPUT_DIR / "smote_experiment"
NO_AUG_OUTPUT_DIR = OUTPUT_DIR / "no_augmentation_experiment"

# 三个最终模型文件的完整路径
MODEL_PATHS = {
    "WGAN-GP": WGAN_GP_OUTPUT_DIR / "trained_models" / "final_xgboost_wgangp_model.joblib",
    "SMOTE": SMOTE_OUTPUT_DIR / "trained_models" / "final_xgboost_smote_model.joblib",
    "No Augmentation": NO_AUG_OUTPUT_DIR / "trained_models" / "final_xgboost_no_aug_model.joblib"
}

# 图表保存路径
OUTPUT_PLOT_PATH = OUTPUT_DIR / "comparison_plots"
os.makedirs(OUTPUT_PLOT_PATH, exist_ok=True)

TARGET_COLUMN = 'target'

# --- 1. 加载数据和模型 ---
print("--- [步骤 1] 加载模型和特征名 ---")

# 加载原始开发集以获取特征名称
try:
    development_df_original = pd.read_excel(DEV_SET_FILE)
    feature_names = development_df_original.drop(columns=[TARGET_COLUMN]).columns.tolist()
    print("成功加载特征名。")
except FileNotFoundError as e:
    print(f"错误: 开发集文件未找到，无法获取特征名。 {e}")
    exit()

# 加载三个模型
models = {}
for name, path in MODEL_PATHS.items():
    try:
        models[name] = joblib.load(path)
        print(f"成功加载模型: {name}")
    except FileNotFoundError:
        print(f"错误: 未找到模型文件 '{path}'。请先运行对应的训练脚本。")
        exit()

# --- 2. 提取并整合特征重要性 ---
print("\n--- [步骤 2] 提取并整合特征重要性 (Gain) ---")

all_importances = []

for name, model in models.items():
    # 使用 get_booster().get_score() 方法来获取指定类型的特征重要性
    # 'gain' 表示特征在所有树中分裂时带来的平均增益
    gain_importance = model.get_booster().get_score(importance_type='gain')
    
    # 将结果转换为Pandas DataFrame
    importance_df = pd.DataFrame({
        'Feature': list(gain_importance.keys()),
        'Gain': list(gain_importance.values())
    })
    
    # 添加一列来标识模型类型
    importance_df['Model'] = name
    all_importances.append(importance_df)

# 将所有结果合并到一个大的DataFrame中
combined_importance_df = pd.concat(all_importances, ignore_index=True)

# 数据透视，方便查看和排序
pivot_df = combined_importance_df.pivot(index='Feature', columns='Model', values='Gain').fillna(0)

# 根据“无增强”模型的Gain值对特征进行降序排序
pivot_df = pivot_df.sort_values(by='No Augmentation', ascending=False)

print("\n--- 特征重要性 (Gain) 对比表 ---")
print(pivot_df)

# --- 3. 可视化对比结果 ---
print("\n--- [步骤 3] 可视化对比结果 ---")

# 我们只可视化最重要的前10个特征（基于无增强模型）
top_10_features = pivot_df.head(10)

# 准备绘图
top_10_features.plot(
    kind='bar',
    figsize=(14, 8),
    width=0.8
)

# plt.title('Top 10 Feature Importances (Gain) Comparison', fontsize=16)
plt.ylabel('Average Gain', fontsize=15)
plt.xlabel('Features', fontsize=15)
plt.xticks(rotation=45, ha='right',fontsize=13) # 旋转X轴标签以防重叠
plt.yticks(fontsize=13)
plt.legend(title='Model')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout() # 调整布局以确保所有元素都可见

# 保存图表
plot_path = OUTPUT_PLOT_PATH / "feature_importance_gain_comparison.png"
plt.savefig(plot_path, dpi=300)
print(f"\n对比图已保存到: {plot_path}")

plt.show()

print("\n--- 特征重要性分析完毕 ---")
